In [ ]:
library('clusterProfiler')
library(org.Mm.eg.db)
library(ggplot2)

ERROR: Error in library(org.Hs.eg.db): there is no package called ‘org.Hs.eg.db’


In [4]:
df.deg <- read.csv("/data2st1/junyi/output/atac0416/dar/region_nt/df_annotattion_wilcoxon.csv")

In [15]:
head(df.deg)

,names,scores,logfoldchanges,pvals,pvals_adj,pct_nz_group,pct_nz_reference,ctname,region,condition,gene_name,gene_id,gstart,gend,strand,distance
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<int>
1,chr2:98666917-98667418,23.533844,0.9004145,1.837545e-122,2.519275e-116,0.9574536,0.8823978,PFC_GABA,PFC,MC,Gm10800,ENSMUSG00000075014.1,98666546,98667301,-,0
2,chr2:98662045-98662546,19.915870,0.9123986,2.964556e-88,2.032203e-82,0.7962252,0.6446533,PFC_GABA,PFC,MC,Gm10801,ENSMUSG00000075015.3,98662236,98664083,+,0
3,chr2:98666414-98666915,18.153790,0.7490824,1.198400e-73,5.476689e-68,0.8691619,0.7602235,PFC_GABA,PFC,MC,Gm10800,ENSMUSG00000075014.1,98666546,98667301,-,0
4,chr9:35305176-35305677,13.207377,0.7783439,7.954764e-40,2.726495e-34,0.5758157,0.4462789,PFC_GABA,PFC,MC,Gm33838,ENSMUSG00000111059.1,35301721,35307599,-,0
5,chr2:98662628-98663129,11.916548,0.7575722,9.704762e-33,2.661046e-27,0.5201536,0.3916688,PFC_GABA,PFC,MC,Gm10801,ENSMUSG00000075015.3,98662236,98664083,+,0
6,chr9:3001884-3002385,8.819262,0.9723987,1.152170e-18,2.632707e-13,0.3000640,0.1894844,PFC_GABA,PFC,MC,Gm10722,ENSMUSG00000091028.1,3000921,3002330,+,0


In [ ]:
# 获取所有唯一的ct_name和condition组合
ct.names <- c("AMY_Glut", "AMY_GABA", "HIP_Glut", "HIP_GABA", "PFC_Glut", "PFC_GABA")
conditions <- c("MC", "MW")

outfolder <- "/data2st1/junyi/output/atac0416/dar/region_nt/"


In [ ]:
# 转换ID（Symbol → Entrez）

# ctname <- "PFC_Glut"
# condition <- "MW"

for (ctname in ct.names) {
  for (condition in conditions) {
    # 筛选数据
    tryCatch(
      {
        print(paste("Processing:", ctname, condition))

        subset_df <- df.deg[df.deg$ctname == ctname & df.deg$condition == condition, ]

        print(paste("Number of genes:", nrow(subset_df)))

        gene_entrez <- bitr(unique(subset_df$gene_name), 
                            fromType = "SYMBOL", 
                            toType = "ENTREZID", 
                            OrgDb = "org.Mm.eg.db")
                            
        entrez_ids <- gene_entrez$ENTREZID
        ego <- enrichGO(gene = entrez_ids,
                        OrgDb = org.Mm.eg.db,
                        keyType = "ENTREZID",
                        ont = "ALL",  # "BP", "MF", "CC" 或 "ALL"
                        pvalueCutoff = 0.05,
                        qvalueCutoff = 0.05,
                        readable = TRUE)  # 结果中显示基因符号

        # 查看结果
        write.csv(ego, file = paste(outfolder, "GO_enrichment_results",ctname,condition,".csv",sep = "_"), row.names = FALSE)
        p <- dotplot(ego, showCategory = 30)
        ggsave(paste(outfolder, "GO_enrichment_dot",ctname,condition,".pdf",sep = "_"), plot = p, width = 12, height = 15)  # 宽度12英寸，高度15英寸

        p<-barplot(ego, showCategory = 30)  # 条形图
        ggsave(paste(outfolder, "GO_enrichment_bar",ctname,condition,".pdf",sep = "_"), plot = p, width = 12, height = 15)  # 宽度12英寸，高度15英寸

        kk <- enrichKEGG(gene = entrez_ids,
                        organism = "mmu",  # 小鼠KEGG代码
                        pvalueCutoff = 0.05,
                        qvalueCutoff = 0.05)

        # 可视化
        p<-dotplot(kk, showCategory = 30)
        write.csv(kk, file = paste(outfolder, "KEGG_enrichment_results",ctname,condition,".csv",sep = "_"), row.names = FALSE)
        ggsave(paste(outfolder, "kegg_enrichment_dot",ctname,condition,".pdf",sep = "_"), plot = p, width = 12, height = 15)  # 宽度12英寸，高度15英寸
      }, 
      error = function(e) {
        print(paste("Error in subsetting data for", ctname, condition))
    })

  }
}

[1] "Processing: AMY_Glut MC"
[1] "Number of genes: 278"


'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(unique(subset_df$gene_name), fromType = "SYMBOL", toType = "ENTREZID", :
“6.91% of input gene IDs are fail to map...”
Warning message in rep(yes, length.out = len):
“'x' is NULL so the result will be NULL”


[1] "Error in subsetting data for AMY_Glut MC"
[1] "Processing: AMY_Glut MW"
[1] "Number of genes: 77"


'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(unique(subset_df$gene_name), fromType = "SYMBOL", toType = "ENTREZID", :
“25.76% of input gene IDs are fail to map...”
Warning message in rep(yes, length.out = len):
“'x' is NULL so the result will be NULL”


[1] "Error in subsetting data for AMY_Glut MW"
[1] "Processing: AMY_GABA MC"
[1] "Number of genes: 442"


...

'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(unique(subset_df$gene_name), fromType = "SYMBOL", toType = "ENTREZID", :
“15.23% of input gene IDs are fail to map...”


[1] "Processing: PFC_GABA MC"
[1] "Number of genes: 12"


'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(unique(subset_df$gene_name), fromType = "SYMBOL", toType = "ENTREZID", :
“60% of input gene IDs are fail to map...”


[1] "Processing: PFC_GABA MW"
[1] "Number of genes: 1473"


'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(unique(subset_df$gene_name), fromType = "SYMBOL", toType = "ENTREZID", :
“7.46% of input gene IDs are fail to map...”
